In [1]:
import sys
import os

# Get the current working directory
current_dir = os.getcwd()

# Add the parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.insert(0, parent_dir)
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from urllib.request import urlretrieve
from scipy.stats import pearsonr

from config.config import DATA_PATH
pd.set_option('display.max_columns', None)

In [2]:
X_train = pd.read_csv(DATA_PATH + '/modeling/X_train.csv')
X_test = pd.read_csv(DATA_PATH + '/modeling/X_test.csv')
y_train = pd.read_csv(DATA_PATH + '/modeling/y_train.csv')
y_test = pd.read_csv(DATA_PATH + '/modeling/y_test.csv')

In [3]:
y_train = y_train['cancelled']
y_test = y_test['cancelled']

In [4]:
X_train.head()

quarter     month  day_of_month  day_of_week  hour_of_day  \
0 -1.361845 -1.620543     -1.452345    -0.984862     0.189847   
1 -0.462343 -0.450140      1.515007    -1.485173    -0.797123   
2  0.437160  0.720262      0.031331     1.016382     0.979423   
3  1.336662  1.012863     -1.452345    -0.984862    -0.402335   
4 -1.361845 -1.327942      1.172621     1.516693     0.979423   

   scheduled_elapsed_time  distance  dep_delay_mean_10D  dep_delay_median_10D  \
0               -0.031459  1.937362            0.617993              2.272436   
1               -0.935554 -1.309607           -0.714057             -0.334788   
2               -0.067265 -0.193030            0.215444              0.056296   
3               -0.774428 -1.139360           -0.896203             -0.660691   
4               -0.872894 -1.351018            0.966584              0.871053   

   dep_delay_max_10D  arr_delay_median_10D  cancelled_sum_10D  \
0          -0.369399              1.263474          -0.320950   
1          -0.688254             -0.571252          -0.320950   
2           0.718457             -0.031627          -0.320950   
3          -0.688254             -0.598233          -0.320950   
4           0.218293              1.884043           1.237885   

   div_airport_landings_sum_10D  n_flights_10D  dep_delay_mean_30D  \
0                      -0.02657      -1.252609            0.753796   
1                      -0.02657       0.282212           -0.720075   
2                      -0.02657       0.172582            0.228256   
3                      -0.02657      -0.485198           -1.015863   
4                      -0.02657      -0.485198           -0.143768   

   dep_delay_median_30D  dep_delay_max_30D  arr_delay_median_30D  \
0              2.917779          -0.651917              1.595429   
1             -0.326840          -0.544094             -0.447513   
2              0.078737           0.592533             -0.118006   
3             -0.651302          -0.822635             -0.645217   
4             -0.326840          -0.229612              0.738711   

   cancelled_sum_30D  div_airport_landings_sum_30D  cancelled_sum_90D  \
0          -0.455432                     -0.045443          -0.614674   
1          -0.455432                     -0.045443          -0.247148   
2          -0.455432                     -0.045443          -0.614674   
3          -0.455432                     -0.045443          -0.614674   
4           1.070058                     -0.045443           0.120378   

   div_airport_landings_sum_90D  temperature_2m_origin  precipitation_origin  \
0                     -0.075479               0.942484             -0.059526   
1                     -0.075479               0.140555             -0.059526   
2                     -0.075479               0.241497             -0.059526   
3                     -0.075479               0.348047             -0.059526   
4                     -0.075479              -0.779140             -0.059526   

   snow_depth_origin  weather_code_origin  visibility_origin  \
0          -0.098763            -0.279899          -0.691314   
1          -0.098763            -0.156940          -0.587775   
2          -0.098763            -0.156940          -0.804266   
3          -0.098763            -0.156940          -0.423054   
4          -0.098763            -0.341378           0.682932   

   wind_speed_10m_origin  wind_direction_10m_origin  wind_gusts_10m_origin  \
0               1.486775                  -0.662031               0.439144   
1               0.382257                   1.248121              -0.070664   
2              -0.080100                   0.878091              -0.509665   
3              -0.388337                  -1.282081              -1.019473   
4               0.921673                   0.378051               1.642857   

   temperature_2m_dest  precipitation_dest  snow_depth_dest  \
0            -0.384728           -0.061437        -0.097708   
1    

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [6]:
logreg_model = LogisticRegression()

logreg_model.fit(X_train, y_train)

y_pred = logreg_model.predict(X_test)

print(classification_report(y_test, y_pred))
print()
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    441319
           1       0.24      0.04      0.06      6416

    accuracy                           0.98    447735
   macro avg       0.61      0.52      0.53    447735
weighted avg       0.98      0.98      0.98    447735


[[440541    778]
 [  6176    240]]


In [7]:
rf_model = RandomForestClassifier()

rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

print(classification_report(y_test, y_pred))
print()
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    441319
           1       0.84      0.15      0.25      6416

    accuracy                           0.99    447735
   macro avg       0.91      0.57      0.62    447735
weighted avg       0.99      0.99      0.98    447735


[[441130    189]
 [  5451    965]]


In [8]:
model_important_features = pd.DataFrame(rf_model.feature_importances_,
                                     columns=['feature_importance'],
                                     index=X_test.columns)
model_important_features = model_important_features.sort_values('feature_importance', ascending=False)
model_important_features.head(20)

feature_importance
cancelled_sum_10D                    0.079835
n_flights_10D                        0.039541
temperature_2m_origin                0.037087
visibility_origin                    0.036816
visibility_dest                      0.036426
day_of_month                         0.035783
temperature_2m_dest                  0.035683
wind_gusts_10m_origin                0.032971
wind_gusts_10m_dest                  0.032191
wind_direction_10m_dest              0.031621
wind_speed_10m_origin                0.031497
wind_direction_10m_origin            0.031200
wind_speed_10m_dest                  0.030346
dep_delay_mean_10D                   0.029388
dep_delay_mean_30D                   0.029068
cancelled_sum_30D                    0.027772
arr_delay_median_10D                 0.026911
dep_delay_max_10D                    0.026773
dep_delay_max_30D                    0.025775
scheduled_elapsed_time               0.025565

In [ ]:
gb_model = GradientBoostingClassifier()

gb_model.fit(X_train, y_train)

y_pred = gb_model.predict(X_test)

print(classification_report(y_test, y_pred))
print()
print(confusion_matrix(y_test, y_pred))

In [62]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, average_precision_score
from sklearn.model_selection import KFold, GridSearchCV
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

In [74]:
model_results = []

def evaluate_model(model, X_train, y_train, X_test, y_test):
    # Fit the model
    model_fit = model.fit(X_train, y_train)
    
    # Predict
    y_pred = model_fit.predict(X_test)
    y_pred_proba = model_fit.predict_proba(X_test)[:, 1]

    # calculate metrics
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    auprc = average_precision_score(y_test, y_pred_proba)
    
    # Return the metrics
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'roc_auc': roc_auc,
        'auprc': auprc
    }

In [75]:
def grid_search_model(model, params, X_train, y_train):

    kf = KFold(n_splits=3, shuffle=True, random_state=40)

    gs = GridSearchCV(model, param_grid=params, cv=kf, scoring='f1', n_jobs=-1)

    gs_fit = gs.fit(X_train, y_train)

    best_estimator, best_params = gs_fit.best_estimator_, gs_fit.best_params_

    return best_estimator, best_params

In [76]:
from sklearn.model_selection import RandomizedSearchCV

def random_grid_search_model(model, params, X_train, y_train):

    kf = KFold(n_splits=3, shuffle=True, random_state=40)

    rs  = RandomizedSearchCV(model, params, n_iter=50, n_jobs=-1, cv=kf)

    rs_fit = rs.fit(X_train, y_train)

    best_estimator, best_params = rs_fit.best_estimator_, rs_fit.best_params_

    return best_estimator, best_params

In [ ]:
log_reg_param_grid = {
   # 'penalty': ['l1', 'l2'],
    'C': np.logspace(-2, 2, 10),
    'solver': ['saga', 'newton-cholesky'], 
    'max_iter': [700],
}

best_model, best_params = grid_search_model(model=LogisticRegression(), 
                                            params=log_reg_param_grid, 
                                            X_train=X_train, 
                                            y_train=y_train)

logreg_metrics = evaluate_model(best_model, X_train, y_train, X_test, y_test)
logreg_metrics['model'] = 'LogisticRegression_GS'
logreg_metrics['best_params'] = best_params
model_results.append(logreg_metrics)

In [19]:
results_df = pd.DataFrame(model_results)
results_df.sort_values('recall', ascending=False)
results_df

accuracy  precision    recall        f1   roc_auc   auprc  \
0  0.984486   0.237103  0.037251  0.064386  0.906372  0.1658   

                   model                                     best_params  
0  LogisticRegression_GS  {'max_iter': 700, 'solver': 'newton-cholesky'}

In [ ]:
rf_param_grid = {
    'n_estimators':[100, 200, 350],
    'criterion': ['gini', 'entropy'],
    'max_depth':[11, 20, None],
    'max_features':['sqrt', 'log2', None],
    'class_weight': [None, 'balanced', 'balanced_subsample']}

best_model, best_params = random_grid_search_model(RandomForestClassifier(), params=rf_param_grid, X_train=X_train, y_train=y_train)
rf_metrics = evaluate_model(best_model, X_train, y_train, X_test, y_test)
rf_metrics['model'] = 'RandomForest_GS'
rf_metrics['best_params'] = best_params
model_results.append(rf_metrics)

In [ ]:
gb_param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'min_samples_split': [5, 10, 15],
    'min_samples_leaf': [1, 5, 10],
    'subsample': [0.8, 1.0],
    'max_features': ['sqrt', 'log2', None]}

best_model, best_params = random_grid_search_model(GradientBoostingClassifier(), gb_param_grid, X_train, y_train)
gs_metrics = evaluate_model(best_model, X_train, y_train, X_test, y_test)
gs_metrics['model'] = 'GradientBoosting_GS'
gs_metrics['best_params'] = best_params
model_results.append(gs_metrics)

In [77]:
models = {
    'RandomForest': RandomForestClassifier(random_state=40),
    'GradientBoosting': GradientBoostingClassifier(random_state=40),
}

for model_name, model in models.items():
    metrics = evaluate_model(model, X_train, y_train, X_test, y_test)
    metrics['model'] = model_name
    model_results.append(metrics)
    
undersampler = RandomUnderSampler(random_state=40)
X_train_res, y_train_res = undersampler.fit_resample(X_train, y_train)

models = {
    'RandomForest_undersampler': RandomForestClassifier(random_state=40),
    'GradientBoosting_undersampler': GradientBoostingClassifier(random_state=40),
}

for model_name, model in models.items():
    metrics = evaluate_model(model, X_train_res, y_train_res, X_test, y_test)
    metrics['model'] = model_name
    model_results.append(metrics)

In [79]:
sm = SMOTE(random_state=40)
X_train_smote, y_train_smote = sm.fit_resample(X_train, y_train)

models = {
    'RandomForest_smote': RandomForestClassifier(random_state=40),
    'GradientBoosting_smote': GradientBoostingClassifier(random_state=40),
}

for model_name, model in models.items():
    metrics = evaluate_model(model, X_train_smote, y_train_smote, X_test, y_test)
    metrics['model'] = model_name
    model_results.append(metrics)



KeyboardInterrupt: 

In [78]:
results_df = pd.DataFrame(model_results)
results_df.sort_values('recall', ascending=False)

accuracy  precision    recall        f1   roc_auc     auprc  \
3  0.877115   0.104357  0.999065  0.188974  0.970835  0.349295   
2  0.877535   0.104612  0.998286  0.189379  0.968297  0.284243   
0  0.987356   0.841012  0.145106  0.247508  0.973540  0.468712   
1  0.986438   0.715000  0.089152  0.158537  0.970147  0.364382   

                           model  
3  GradientBoosting_undersampler  
2      RandomForest_undersampler  
0                   RandomForest  
1               GradientBoosting

# PCA

In [63]:
from sklearn.decomposition import PCA

In [64]:
cat_cols = ['diverted', 'code_share_flight', 'is_holiday', 'dep_window_early afternoon', 'dep_window_early morning', 'dep_window_evening', 'dep_window_late night', 'dep_window_midday', 'dep_window_morning',	'dep_window_night',	'dep_window_overnight']

In [65]:
X_train

quarter     month  day_of_month  day_of_week  hour_of_day  \
0       -1.361845 -1.620543     -1.452345    -0.984862     0.189847   
1       -0.462343 -0.450140      1.515007    -1.485173    -0.797123   
2        0.437160  0.720262      0.031331     1.016382     0.979423   
3        1.336662  1.012863     -1.452345    -0.984862    -0.402335   
4       -1.361845 -1.327942      1.172621     1.516693     0.979423   
...           ...       ...           ...          ...          ...   
1044707  0.437160  0.135061     -0.767571     1.516693     0.584635   
1044708 -1.361845 -1.035342      1.629136     0.015760    -1.389305   
1044709  0.437160  0.720262     -1.566474     1.016382    -0.007547   
1044710  1.336662  1.012863      0.601976     1.016382    -1.389305   
1044711  0.437160  0.427661      0.487847     1.516693     1.176817   

         scheduled_elapsed_time  distance  dep_delay_mean_10D  \
0                     -0.031459  1.937362            0.617993   
1                     -0.935554 -1.309607           -0.714057   
2                     -0.067265 -0.193030            0.215444   
3                     -0.774428 -1.139360           -0.896203   
4                     -0.872894 -1.351018            0.966584   
...                         ...       ...                 ...   
1044707               -0.344759 -0.349473            0.059701   
1044708               -0.944505 -1.369423           -0.396804   
1044709               -0.120974 -0.205300           -0.246678   
1044710               -0.747574 -1.035064           -0.759353   
1044711               -0.989262 -1.265128           -0.094948   

         dep_delay_median_10D  dep_delay_max_10D  arr_delay_median_10D  \
0                    2.272436          -0.369399              1.263474   
1                   -0.334788          -0.688254             -0.571252   
2                    0.056296           0.718457             -0.031627   
3                   -0.660691          -0.688254             -0.598233   
4                    0.871053           0.218293              1.884043   
...                       ...                ...                   ...   
1044707              0.382199          -0.225602              0.076298   
1044708             -0.530330           0.312074             -0.112571   
1044709             -0.171836          -0.431920             -0.652196   
1044710             -0.465149          -0.513196             -0.787102   
1044711              0.479970          -0.400659              0.427055   

         cancelled_sum_10D  div_airport_landings_sum_10D  n_flights_10D  \
0                -0.320950                      -0.02657      -1.252609   
1                -0.320950                      -0.02657       0.282212   
2                -0.320950                      -0.02657       0.172582   
3                -0.320950                      -0.02657      -0.485198   
4                 1.237885                      -0.02657      -0.485198   
...                    ...                           ...            ...   
1044707          -0.320950                      -0.02657      -0.485198   
1044708          -0.320950                      -0.02657       2.803702   
1044709          -0.320950                      -0.02657       0.611102   
1044710          -0.320950                      -0.02657       0.611102   
1044711           2.796720                      -0.02657      -0.046678   

         dep_delay_mean_30D  dep_delay_median_30D  dep_delay_max_30D  \
0                  0.753796              2.917779          -0.651917   
1                 -0.720075             -0.326840          -0.544094   
2                  0.228256              0.078737           0.592533   
3                 -1.015863             -0.651302          -0.822635   
4                 -0.143768             -0.326840          -0.229612   
...                     ...                   ...                ...   
1044707            0.763938              0.565430          -0.126283   
1044708   

In [66]:
X_train_num = X_train.drop(columns=cat_cols)
X_test_num = X_test.drop(columns=cat_cols)

In [67]:
pca = PCA(n_components=0.95)
pca_fit = pca.fit(X_train_num)
X_train_num_pca = pca_fit.transform(X_train_num)
X_test_num_pca = pca_fit.transform(X_test_num)

In [68]:
features = range(pca_fit.n_components_)
plt.bar(features, pca_fit.explained_variance_ratio_)
plt.xlabel('Number of Components')
plt.ylabel('Explained Variance Ratio')
plt.title('Explained Variance by Number of Components')
plt.show()

In [71]:
cols = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17', 'PC18', 'PC19', 'PC20', 'PC21', 'PC22', 'PC23', 'PC24', 'PC25', 'PC26', 'PC27', 'PC28', 'PC29', 'PC30', 'PC31']
loadings = pd.DataFrame(pca_fit.components_.T, columns=cols, index=X_train_num.columns)
loadings.sort_values('PC1', ascending=False)

PC1           PC2           PC3  \
dep_delay_mean_30D            0.386403  3.325652e-02 -1.240314e-02   
dep_delay_mean_10D            0.371403  2.997176e-02 -1.208310e-02   
dep_delay_median_10D          0.365193  8.959683e-02 -1.559488e-02   
dep_delay_median_30D          0.360581  9.652989e-02 -1.692761e-02   
arr_delay_median_10D          0.357945  1.011903e-01  6.660682e-04   
arr_delay_median_30D          0.355968  1.125866e-01  1.269579e-03   
dep_delay_max_10D             0.209502 -5.408873e-02 -8.442015e-03   
dep_delay_max_30D             0.177094 -6.761551e-02 -9.796522e-03   
hour_of_day                   0.162497  7.847378e-02  1.617299e-03   
temperature_2m_origin         0.118288  1.004137e-01  2.041319e-02   
cancelled_sum_30D             0.117485 -2.511560e-01 -2.121255e-04   
cancelled_sum_10D             0.102296 -1.979881e-01  5.186561e-04   
airline_mkt_encoded           0.093984 -1.821670e-01  3.071133e-02   
cancelled_sum_90D             0.085722 -2.735446e-01 -2.442442e-03   
temperature_2m_dest           0.072198  7.059481e-02 -1.025627e-01   
airline_ops_encoded           0.070621 -2.567366e-01  2.165223e-02   
wind_speed_10m_origin         0.052839 -1.070204e-01 -9.797767e-02   
dest_encoded                  0.046658 -2.529065e-01  4.017959e-01   
dest_division_encoded         0.043368 -2.542422e-01  4.250228e-01   
dest_region_encoded           0.041531 -2.605815e-01  4.145143e-01   
visibility_origin             0.036771  9.582192e-02  7.867611e-03   
origin_encoded                0.036564 -3.324051e-01 -3.146868e-01   
wind_gusts_10m_origin         0.028348 -1.256994e-01 -9.960369e-02   
weather_code_origin           0.025314 -2.932667e-02 -1.134598e-02   
origin_division_encoded       0.023079 -3.355663e-01 -3.378576e-01   
origin_region_encoded         0.018112 -3.397112e-01 -3.235860e-01   
precipitation_origin          0.014955 -1.532112e-02 -9.693316e-03   
div_airport_landings_sum_30D  0.012920 -2.898359e-02  9.786740e-03   
div_airport_landings_sum_90D  0.012241 -2.839969e-02  1.080740e-02   
weather_code_dest             0.011712 -3.440346e-02  2.509396e-02   
div_airport_landings_sum_10D  0.011080 -2.219651e-02  7.088898e-03   
precipitation_dest            0.009700 -1.519526e-02  1.367105e-02   
wind_gusts_10m_dest           0.009666 -1.191940e-01  1.392918e-01   
wind_speed_10m_dest           0.002823 -1.144604e-01  1.205007e-01   
day_of_week                   0.001316  1.520300e-02  1.632206e-04   
wind_direction_10m_dest       0.000941 -1.118478e-02  2.443668e-02   
snowfall_origin              -0.000000  1.262177e-29  6.462349e-27   
snowfall_dest                -0.000000 -4.900838e-31 -2.628323e-28   
snow_depth_origin            -0.001173 -8.495440e-03  1.008403e-02   
visibility_dest              -0.002912  7.482887e-02 -6.330098e-02   
distance                     -0.005041  7.136682e-02  9.005477e-02   
scheduled_elapsed_time       -0.005453  5.274981e-02  2.936685e-01   
snow_depth_dest              -0.005651 -8.999751e-03  2.939677e-03   
day_of_month                 -0.011479 -9.158647e-03  9.928182e-04   
wind_direction_10m_origin    -0.012624 -2.677551e-02 -1.907393e-02   
quarter                      -0.055593  7.141041e-02 -1.736394e-02   
month                        -0.062924  7.063049e-02 -1.668233e-02   
n_flights_10D                -0.078903 -1.566457e-01 -6.091938e-03   

                                       PC4           PC5           PC6  \
dep_delay_mean_30D           -1.659516e-02  1.164587e-02 -4.544937e-02   
dep_delay_mean_10D           -2.380865e-02  4.591276e-02 -7.772816e-02   
dep_delay_median_10D          5.338638e-02 -6.549566e-02  3.488303e-02   
dep_delay_median_30D          5.668690e-02 -7.559701e-02  4.493780e-02   
arr_delay_median_10D          6.352240e-02 -2.673832e-02  2.969279e-02   
arr_delay_median_30D          6.520030e-02 -3.593675e-02  4.267537e-02   
dep_delay_max_10D            -1.248242e-01  1.927081e-01 -1.852240e-01   
dep_delay_ma

In [72]:
X_train_final = np.concatenate([X_train_num_pca, X_train[cat_cols].values], axis=1)
X_test_final = np.concatenate([X_test_num_pca, X_test[cat_cols].values], axis=1)

In [73]:
logreg_model = LogisticRegression()

logreg_model.fit(X_train_final, y_train)

y_pred = logreg_model.predict(X_test_final)

print(classification_report(y_test, y_pred))
print()
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    441319
           1       0.24      0.04      0.07      6416

    accuracy                           0.98    447735
   macro avg       0.61      0.52      0.53    447735
weighted avg       0.98      0.98      0.98    447735


[[440564    755]
 [  6173    243]]


In [52]:
rf_model = RandomForestClassifier()

rf_model.fit(X_train_final, y_train)

y_pred = rf_model.predict(X_test_final)

print(classification_report(y_test, y_pred))
print()
print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.99      1.00      0.99    441319
           1       0.76      0.02      0.04      6416

    accuracy                           0.99    447735
   macro avg       0.87      0.51      0.52    447735
weighted avg       0.98      0.99      0.98    447735


[[441275     44]
 [  6274    142]]


In [ ]:
model_important_features = pd.DataFrame(rf_model.feature_importances_,
                                     columns=['feature_importance'],
                                     index=X_test_final.columns)
model_important_features = model_important_features.sort_values('feature_importance', ascending=False)
model_important_features.head(20)

In [ ]:
gb_model = GradientBoostingClassifier()

gb_model.fit(X_train, y_train)

y_pred = gb_model.predict(X_test)

print(classification_report(y_test, y_pred))
print()
print(confusion_matrix(y_test, y_pred))

In [45]:
from sklearn.feature_selection import SelectKBest, f_classif

selector = SelectKBest(f_classif, k=30)
selector.fit(X_train, y_train)
X_train_reduced = selector.transform(X_train)

selector.get_feature_names_out

/Users/katialopes-gilbert/opt/anaconda3/envs/data-course/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [36 45 60] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/Users/katialopes-gilbert/opt/anaconda3/envs/data-course/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


<bound method SelectorMixin.get_feature_names_out of SelectKBest(k=30)>